<a href="https://colab.research.google.com/github/ClaudioMSOliveira/BibliotecaDev/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%pip -q install google-genai

In [10]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [11]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [12]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no [Instagram](https://www.instagram.com/aluraonline/), [LinkedIn](https://www.linkedin.com/company/alura/), e [Twitter](https://twitter.com/AluraOnline). Eles costumam anunciar novos cursos e imersões por lá.
*   **Verificar a página de Imersões da Alura:** Visite a página de Imersões da Alura no site deles ([https://www.alura.com.br/imersao](https://www.alura.com.br/imersao)). Lá você encontrará informações sobre as imersões atuais e futuras.
*   **Assinar a newsletter da Alura:** Ao assinar a newsletter da Alura, você receberá informações sobre lançamentos de cursos, promoções e eventos, incluindo as Imersões.

In [13]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio de 2025.

In [14]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: starten.tech, youtube.com, alura.com.br



In [15]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [16]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [17]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [18]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [28]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (Google Search) para recuperar as últimas
        notícias de lançamentos muito relevante sobre o tópico especificado. Foque em no máximo 5 notícias com base na quantidade e
        entusiasmo das notícias sobre o tópico. Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja
        tão relevante assim e pode ser substituído por outro que tenha mais. Esses lançamentos relevantes devem ser atuais de no máximo
        um mês antes da data de hoje.
        """,
        description="Agente que busca notícias no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [20]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lançamentos mais recentes buscados, você deve:
        - usar o Google Search para criar um plano sobre quais são os pontos mais relevantes que poderíamos abordar em um post sobre cada
          um deles.
        - usar o Google Search para encontrar mais informações sobre os temas e aprofundar o assunto.
        - ao final, você irá escolhar o tema mais relevante com base nas suas pesquisas e retorná-lo com seus pontos relevantes e um plano com
        os assuntos a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [32]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [33]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [34]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Por favor, insira um tópico válido.")
else:
    print(f"Você quer novidades sobre o seguinte tema: {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- Resultados do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("----------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- Resultados do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("----------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- Resultados do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("----------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- Resultados do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("----------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: agentes de IA
Você quer novidades sobre o seguinte tema: agentes de IA

--- Resultados do Agente 1 (Buscador) ---



> Aqui estão algumas notícias recentes sobre agentes de IA, coletadas a partir da minha pesquisa no Google:
> 
>  
> Com base nas notícias mais recentes sobre agentes de IA:
> 
> 1.  **Avanço dos Agentes de IA:** Em 12 de março de 2025, o IT Forum publicou um artigo destacando 2025 como o ano em que os agentes de IA estão saindo dos bastidores, com investimentos bilionários e aplicações crescentes em setores como e-commerce, atendimento ao cliente, marketing e finanças.
> 2.  **Impacto no Comércio:** Um relatório da Visa e do Institute for the Future (IFTF), divulgado em 15 de maio de 2025, aponta que agentes de IA vão alterar drasticamente a forma como compramos, com 82% dos brasileiros demonstrando interesse em assistentes pessoais com tecnologia de IA.
> 3.  **Funcionalidade e Exemplos:** A CNN Brasil publicou um artigo em 21 de março de 2025, explicando como os agentes de IA funcionam e podem ajudar na rotina, citando exemplos como robo-advisors e o Browser Operator da Opera, que automatiza tarefas de navegação.
> 4.  **Desafios e Definições:** Em 13 de maio de 2025, o Startups publicou um artigo mencionando que nem investidores sabem ao certo o que são agentes de IA, destacando a necessidade de uma inteligência geral artificial (AGI) para operações autônomas de longo prazo.
> 5.  **Automação do Trabalho:** A Verx publicou um artigo em 16 de maio de 2025, sobre agentes de IA e a automação do trabalho, explorando se são uma ameaça ou oportunidade, e como empresas podem se adaptar para aproveitar ao máximo as tecnologias emergentes.


----------------------------------------------------

--- Resultados do Agente 2 (Planejador) ---



> Ok, com base nessas notícias recentes sobre agentes de IA, vou criar um plano de conteúdo explorando os pontos mais relevantes para um post.
> 
> **Pesquisas no Google para aprofundar os temas:**
> 
> 
> Com base nas pesquisas, o tema mais relevante para o post é:
> 
> **Agentes de IA: Ameaça ou Oportunidade no Mercado de Trabalho e no Comércio?**
> 
> **Pontos Relevantes:**
> 
> *   **Definição e Funcionalidade:** Explicar o que são agentes de IA, como funcionam e exemplos práticos (robo-advisors, Browser Operator da Opera, chatbots, etc.).
> *   **Aplicações Atuais e Futuras:** Explorar as áreas onde os agentes de IA estão sendo aplicados (e-commerce, atendimento ao cliente, marketing, finanças, saúde, logística, etc.) e como estão transformando esses setores. Mencionar exemplos de uso na B3 (mercado financeiro).
> *   **Impacto no Comércio:** Discutir como os agentes de IA estão mudando a forma como compramos, o interesse dos brasileiros em assistentes pessoais com IA e a importância da confiança e transparência nesse novo cenário (relatório Visa/IFTF).
> *   **Automação do Trabalho:** Abordar a questão da automação do trabalho, se os agentes de IA são uma ameaça ou oportunidade, e como empresas e profissionais podem se adaptar para aproveitar ao máximo essas tecnologias.
> *   **IA vs. AGI:** Explicar a diferença entre IA (Inteligência Artificial) e AGI (Inteligência Artificial Geral), destacando que a AGI ainda é um objetivo futuro e a necessidade de uma AGI para operações autônomas de longo prazo.
> *   **Investimentos e Crescimento:** Mencionar os investimentos bilionários em IA e o crescimento do mercado de agentes de IA, tanto globalmente quanto no Brasil.
> *   **Desafios e Considerações Éticas:** Discutir os desafios, como a falta de clareza sobre o que são agentes de IA, a necessidade de segurança e privacidade, e as considerações éticas no desenvolvimento de máquinas superinteligentes.
> *   **Oportunidades para Programadores:** Explorar como os agentes de IA podem criar novas oportunidades para programadores, como automação de tarefas repetitivas, desenvolvimento de novas ferramentas e sistemas, e a necessidade de supervisão humana e conhecimento do domínio.
> 
> **Plano do Post:**
> 
> 1.  **Introdução:**
>     *   Apresentar o tema dos agentes de IA como protagonistas da transformação digital em 2025.
>     *   Destacar a crescente relevância dos agentes de IA e os investimentos bilionários na área.
>     *   Levantar a questão central: agentes de IA são uma ameaça ou uma oportunidade?
> 2.  **O que são Agentes de IA?**
>     *   Definir agentes de IA de forma clara e acessível.
>     *   Exemplos práticos de agentes de IA em uso atualmente.
> 3.  **Aplicações e Impacto nos Setores:**
>     *   E-commerce: Personalização da experiência de compra, automação de campanhas.
>     *   Atendimento ao cliente: Chatbots inteligentes, respostas rápidas e eficientes.
>     *   Finanças: Robo-advisors, análise preditiva, gestão de fraudes.
>     *   Saúde: Diagnósticos assistidos, triagem automatizada.
>     *   Logística: Controle de frotas, otimização de rotas.
> 4.  **Agentes de IA no Comércio: A Revolução nas Compras:**
>     *   Relatório Visa/IFTF: Agentes de IA vão alterar drasticamente a forma como compramos.
>     *   Interesse dos brasileiros em assistentes pessoais com IA.
>     *   A importância da confiança e transparência no comércio com IA.
> 5.  **Ameaça ou Oportunidade no Mercado de Trabalho?**
>     *   Automação de tarefas e substituição de empregos: O lado da ameaça.
>     *   Novas oportunidades: Desenvolvimento, supervisão, manutenção de sistemas de IA.
>     *   Como empresas e profissionais podem se adaptar para aproveitar as tecnologias emergentes.
> 6.  **IA vs. AGI: Qual a Diferença e Por Que Importa?**
>     *   Explicar as diferenças entre IA e AGI.
>     *   A necessidade de AGI para operações autônomas de longo prazo.
> 7.  **Investimentos e o Futuro do Mercado de Agentes de IA:**
>     *   Investimentos bilionários em IA.
>     *   Crescimento do mercado de agentes de IA globalmente e no Brasil.
> 8.  **Desafios e Considerações Éticas:**
>     *   Falta de clareza sobre o que são agentes de IA.
>     *   Necessidade de segurança e privacidade.
>     *   Considerações éticas no desenvolvimento de máquinas superinteligentes.
> 9.  **Conclusão:**
>     *   Reafirmar a importância dos agentes de IA na transformação digital.
>     *   Incentivar a adaptação e o aprendizado contínuo para aproveitar as oportunidades.
>     *   Reforçar a necessidade de um debate ético e responsável sobre o desenvolvimento e uso de agentes de IA.
> 


----------------------------------------------------

--- Resultados do Agente 3 (Redator) ---



> 🚀 Agentes de IA: A revolução chegou e está transformando tudo! 🤖
> 
> Será que eles são a chave para o futuro ou uma ameaça no mercado de trabalho? 🤔 A resposta é: depende de como você se prepara!
> 
> 💡 **O que são Agentes de IA?** São softwares espertos que automatizam tarefas e tomam decisões. Imagina ter um assistente que aprende seus gostos e faz compras por você! 🛍️
> 
> 💼 **Onde eles já estão?** E-commerce, finanças (olha os robo-advisors!), saúde, logística... Eles estão em todo lugar, otimizando processos e abrindo um mundo de possibilidades.
> 
> 💰 **E no mercado de trabalho?** Calma! 🧘‍♀️ A automação pode assustar, mas também cria novas oportunidades incríveis para quem programa, supervisiona e entende do negócio.
> 
> 🔑 **A palavra-chave é ADAPTAÇÃO!** Invista em conhecimento, aprenda sobre IA e esteja pronto para o futuro.
> 
> 🚀 **A Alura te prepara para essa jornada!** 😉
> 
> #AgentesIA #InteligenciaArtificial #Alura #FuturoDoTrabalho
> 


----------------------------------------------------

--- Resultados do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos deixá-lo ainda mais engajador para o público do Instagram, com foco em clareza e dinamismo. Aqui estão algumas sugestões:
> 
> **Problemas e Sugestões:**
> 
> 1.  **Tom:** O tom está bom, mas podemos adicionar mais personalidade e usar uma linguagem ainda mais próxima da audiência jovem.
> 2.  **Emojis:** O uso de emojis está bom, mas podemos diversificar um pouco para deixar o post mais atrativo visualmente.
> 3.  **Chamada para Ação (CTA):** Podemos fortalecer a CTA no final para incentivar mais cliques e engajamento.
> 4.  **Hashtags:** As hashtags estão ok, mas podemos adicionar algumas mais específicas e populares.
> 
> **Rascunho Revisado:**
> 
> 🚀 **Agentes de IA: O futuro chegou pra ficar (e tá transformando TUDO)!** 🤖
> 
> Será que essa galera da IA vai dominar o mundo ou só dar um upgrade no nosso trabalho? 🤔 A real é: depende de você!
> 
> 💡 **Mas, afinal, o que são Agentes de IA?** 🤔 Pensa num software super inteligente que faz tudo no automático e ainda aprende com você! Tipo um assistente pessoal que te conhece melhor que você mesmo e ainda faz as compras do mês! 🤯
> 
> 💼 **Onde eles estão?** No e-commerce, nas finanças (já viu os robo-advisors?), na saúde, na logística… Eles tão ON em tudo, turbinando os processos e criando um montão de oportunidades! 🤩
> 
> 💰 **E o emprego, como fica?** 🧘 Calma, respira! A automação pode parecer assustadora, mas ela também abre portas INCRÍVEIS pra quem manda em programação, entende de negócios e saca tudo de IA. 😉
> 
> 🔑 **A chave? ADAPTAÇÃO!** 🚀 Se liga: Invista em conhecimento, estude IA e prepare-se para o futuro.
> 
> 🔥 **Quer virar expert em Agentes de IA e dominar essa parada? A Alura te dá o mapa do tesouro!** 😉 Clica no link da bio e vem com a gente! 🚀
> 
> #AgentesIA #InteligenciaArtificial #IA #Alura #FuturoDoTrabalho #Tecnologia #Inovação #CarreiraTech
> 


----------------------------------------------------
